In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Начало

In [4]:
train_dataTest = train_data
test_dataTest = test_data
train_dataTest['isTrain'] = 1
test_dataTest['isTrain'] = 0
combined_data = pd.concat([train_dataTest, test_dataTest], sort=False)

# Разбиваем пассажиров на семьи

In [5]:


# Извлечение фамилий из столбца Name
combined_data['LastName'] = combined_data['Name'].apply(lambda name: name.split(',')[0])

# Создание столбца FamilySize (количество членов семьи на борту)
combined_data['FamilySize'] = combined_data['SibSp'] + combined_data['Parch']

# Создание столбца FamilyKey на основе фамилии и билета, если есть родственники
combined_data['FamilyKey'] = combined_data.apply(
    lambda row: f"{row['LastName']}_{row['FamilySize']}" if row['FamilySize'] > 0 else 'NoFamily', axis=1)
#print(FamilyKey)
# Присваивание уникального целочисленного FamilyID для каждой семьи
# Используем метод factorize для преобразования строковых значений в целые числа

family_mapping = {key: idx + 1 for idx, key in enumerate(combined_data['FamilyKey'].unique())}
combined_data['FamilyID'] = combined_data['FamilyKey'].map(family_mapping).fillna(0).astype(int) 
# Устанавливаем FamilyID для пассажиров без семьи на 0
combined_data.loc[combined_data['FamilySize'] == 0, 'FamilyID'] = 0

train_dataTest['FamilySize'] = ''
train_dataTest['FamilyID'] = ''




train_dataTest = combined_data[combined_data['isTrain'] == 1].drop(columns=['FamilyKey','LastName'])
test_dataTest = combined_data[combined_data['isTrain'] == 0].drop(columns=['FamilyKey','LastName'])

combined_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isTrain,LastName,FamilySize,FamilyKey,FamilyID
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Braund,1,Braund_1,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Cumings,1,Cumings_1,2
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Heikkinen,0,NoFamily,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Futrelle,1,Futrelle_1,4
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Allen,0,NoFamily,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0,Spector,0,NoFamily,0
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0,Oliva y Ocana,0,NoFamily,0
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0,Saether,0,NoFamily,0
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0,Ware,0,NoFamily,0


# Выделяем из имени титул и заполняем пропущенные значения в Age медианным значением для титула

In [6]:
def extract_title(name):
    if 'Mr.' in name:
        return 1
    elif 'Master.' in name:
        return 2
    elif 'Miss.' in name:
        return 3
    elif 'Mrs.' in name:
        return 4
    else:
        return 0

# Применение функции для создания столбца Title
combined_data['Title'] = combined_data['Name'].apply(extract_title)

median_ages = combined_data.groupby('Title')['Age'].median()

# Функция для заполнения пропусков в Age
def fill_age(row):
    if pd.isnull(row['Age']):
        return median_ages[row['Title']]
    else:
        return row['Age']

# Функция для корректировки значений Age
def fix_age(value):
    if value < 1:  # Проверяем, является ли значение менее 1
        return value * 100  # Преобразуем значение в проценты
    return value    

combined_data['Age'] = combined_data['Age'].apply(fix_age)
# Применение функции для заполнения пропусков
combined_data['Age'] = combined_data.apply(fill_age, axis=1)

# Вывод первых строк с новым столбцом
combined_data.head

<bound method NDFrame.head of      PassengerId  Survived  Pclass  \
0              1       0.0       3   
1              2       1.0       1   
2              3       1.0       3   
3              4       1.0       1   
4              5       0.0       3   
..           ...       ...     ...   
413         1305       NaN       3   
414         1306       NaN       1   
415         1307       NaN       3   
416         1308       NaN       3   
417         1309       NaN       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                     

In [7]:
age_train = combined_data[combined_data['isTrain'] == 1]['Age']
age_test = combined_data[combined_data['isTrain'] == 0]['Age']
train_dataTest['Age'] = age_train
test_dataTest['Age'] = age_test

title_train = combined_data[combined_data['isTrain'] == 1]['Title']
title_test = combined_data[combined_data['isTrain'] == 0]['Title']
train_dataTest['Title'] = title_train
test_dataTest['Title'] = title_test

In [8]:
train_dataTest.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isTrain,FamilySize,FamilyID,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,1,1,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1,2,4
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,0,3
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1,4,4
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0,0,1


# Определяем кол-во выживших членов семьи

In [9]:
# Создаём столбец FamilySurvival для обучающей выборки
family_survival_dict = {}

# Вычисляем количество выживших родственников для каждой семьи в обучающей выборке
for _, row in train_dataTest.iterrows():
    if row['FamilySize'] > 0:
        if row['FamilyID'] not in family_survival_dict:
            family_survival_dict[row['FamilyID']] = 0
        family_survival_dict[row['FamilyID']] += row['Survived']

# Присваиваем FamilySurvival в обучающей выборке
train_dataTest['FamilySurvival'] = train_dataTest.apply(
    lambda row: family_survival_dict.get(row['FamilyID'], 0) if row['FamilySize'] > 0 else 0, axis=1)

# Присваиваем FamilySurvival в тестовой выборке на основе обучающей выборки
test_dataTest['FamilySurvival'] = test_dataTest['FamilyID'].map(family_survival_dict).fillna(0).astype(int)


In [10]:
test_dataTest.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isTrain,FamilySize,FamilyID,Title,FamilySurvival
0,892,NaN,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0,0,1,0
1,893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,1,204,4,0
2,894,NaN,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,0,0,1,0
3,895,NaN,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,0,1,0
4,896,NaN,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,2,205,4,0


# Выделяем 5 категорий билетов по стоимости

In [11]:
# Определение квантилей для столбца Fare (0%, 25%, 50%, 75%, 87.5%, 100%)
fare_bins = pd.qcut(combined_data['Fare'], q=[0, 0.25, 0.5, 0.75, 0.875, 1.0], labels=[0, 1, 2, 3, 4])
combined_data['Fare_Categ'] = fare_bins
combined_data['Fare_Categ'] = combined_data['Fare_Categ'].cat.codes.astype(int)

fare_train = combined_data[combined_data['isTrain'] == 1]['Fare_Categ']
fare_test = combined_data[combined_data['isTrain'] == 0]['Fare_Categ']
train_dataTest['Fare_Categ'] = fare_train
test_dataTest['Fare_Categ'] = fare_test

In [12]:
combined_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isTrain,LastName,FamilySize,FamilyKey,FamilyID,Title,Fare_Categ
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Braund,1,Braund_1,1,1,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Cumings,1,Cumings_1,2,4,4
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Heikkinen,0,NoFamily,0,3,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Futrelle,1,Futrelle_1,4,4,3
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Allen,0,NoFamily,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,29.0,0,0,A.5. 3236,8.0500,NaN,S,0,Spector,0,NoFamily,0,1,1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0,Oliva y Ocana,0,NoFamily,0,0,4
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0,Saether,0,NoFamily,0,1,0
416,1308,NaN,3,"Ware, Mr. Frederick",male,29.0,0,0,359309,8.0500,NaN,S,0,Ware,0,NoFamily,0,1,1


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

y = train_dataTest["Survived"]

features = ["Sex", "FamilySurvival", "Fare_Categ", "Pclass", "Title", "Age"]
train_dataTest
X = pd.get_dummies(train_dataTest[features])
X_test = pd.get_dummies(test_dataTest[features])

model = RandomForestClassifier(n_estimators=145, max_depth=5, random_state=1,
                                min_samples_leaf = 2, min_samples_split = 6)
model.fit(X, y)



predictions = model.predict(X_test)
predictions = [int(x) for x in predictions]
output = pd.DataFrame({'PassengerId': test_dataTest.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
